In [1]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.

GPU run command with Theano backend (with TensorFlow, the GPU is automatically used):
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10_cnn.py

It gets down to 0.65 test logloss in 25 epochs, and down to 0.55 after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from datetime import datetime
from scipy.spatial.distance import pdist

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

%matplotlib inline

Using TensorFlow backend.


In [2]:
nb_classes = 10

# input image dimensions
img_rows, img_cols = 32, 32
# The CIFAR10 images are RGB.
img_channels = 3

# The data, shuffled and split between train and test sets:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
model=load_model('cifar10.h5')
get_13rd_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                   [model.layers[13].output])
get_16rd_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                   [model.layers[16].output])

In [7]:
from collections import defaultdict
from functools import partial
from tqdm import tqdm

batch_size = 10

dist_funcs = ['euclidean', 'cosine', 'correlation']

# http://stackoverflow.com/questions/25014298/creating-a-defaultdict-with-empty-numpy-array/25014320#25014320
i_dist = defaultdict(list)
f_dist = defaultdict(list)
f16_dist = defaultdict(list)
evaluate = []

    
for (zoom, width, height, rotation) in tqdm(zip(np.linspace(0.0,0.4,20),
                                                      np.linspace(0.0,0.4,20),
                                                      np.linspace(0.0,0.4,20),
                                                      np.linspace(0,45,20))):
    batch_idx = 0
    datagen = ImageDataGenerator(zoom_range=zoom,
                                 width_shift_range=width,
                                 height_shift_range=height,
                                 rotation_range=rotation,
                                 horizontal_flip=True)
    
    temp_i_dist = defaultdict(list)
    temp_f_dist = defaultdict(list)
    temp_f16_dist = defaultdict(list)
    temp_evaluate = []

    for X_batch, Y_batch in datagen.flow(np.repeat(X_test, batch_size, axis=0),
                                         np.repeat(Y_test, batch_size, axis=0),
                                         batch_size=batch_size,
                                         shuffle=False):
        for dist_func in dist_funcs:
            X_batch_reshaped = X_batch.reshape(batch_size, -1)
            temp_i_dist[dist_func].append(pdist(X_batch_reshaped, dist_func).tolist())
        
            features = get_13rd_layer_output([X_batch, 0])[0]
            temp_f_dist[dist_func].append(pdist(features, dist_func).tolist())
            
            features = get_16rd_layer_output([X_batch, 0])[0]
            temp_f16_dist[dist_func].append(pdist(features, dist_func).tolist())
        
        temp_evaluate.append(model.evaluate(X_batch, Y_batch, verbose=0)[1])
        
        batch_idx += 1
        if batch_idx >= len(X_test) / batch_size:
            break

    
    
    for dist_func in dist_funcs:
        i_dist[dist_func].append(np.mean(temp_i_dist[dist_func]))
        f_dist[dist_func].append(np.mean(temp_f_dist[dist_func]))
        f16_dist[dist_func].append(np.mean(temp_f16_dist[dist_func]))
    evaluate.append(np.mean(temp_evaluate))

100%|██████████| 20/20 [09:51<00:00, 27.73s/it]


In [ ]:
# fix = []
# for i in range(len(evaluate)):
#     if i % 3 == 0:
#         fix.append(evaluate[i])
# print(len(fix))

In [ ]:
for dist_func in dist_funcs:
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    fig.subplots_adjust(hspace=0.2)
    ax.set_title(dist_func)
    ax.set_ylabel('mean feature distance')
    ax.set_xlabel('mean image distance')

    ax.plot(i_dist[dist_func], f_dist[dist_func], 'ro',label='before last layer (512 features)')
    fit = np.polyfit(i_dist[dist_func],f_dist[dist_func],1)
    fit_fn = np.poly1d(fit) 
    ax.plot(i_dist[dist_func], fit_fn(i_dist[dist_func]), '--r')
    

    
    ax.plot(i_dist[dist_func], f16_dist[dist_func], 'bo', label='last layer (10 features)')
    fit = np.polyfit(i_dist[dist_func],f16_dist[dist_func],1)
    fit_fn = np.poly1d(fit) 
    ax.plot(i_dist[dist_func], fit_fn(i_dist[dist_func]), '--b')
    
    ax2 = ax.twinx()
    ax2.plot(i_dist[dist_func], evaluate, 'go', label='accuracy')
    ax2.set_ylabel('accuracy')
    ax.legend(loc=0)
    ax2.legend(loc=0)
    ax2.set_ylim([0.0,1.0])
    plt.savefig('{}.png'.format(dist_func))

    